In [1]:
# VER RECURSOS DE LA CLASE PARA INSTALAR SCRAPY
from scrapy.item import Field
from scrapy.item import Item
from scrapy.spiders import Spider
from scrapy.selector import Selector
from scrapy.loader.processors import MapCompose
from scrapy.loader import ItemLoader
from bs4 import BeautifulSoup
from scrapy.crawler import CrawlerProcess

In [2]:

A=[]

# Información a guardar
class Pregunta(Item):
    id = Field()
    cargo = Field()
    fecha = Field()
    salario = Field()
    empresa = Field()

# Tipo de Spider
class QuotesSpider(Spider):
    name = "Spider" 
    # Configuración del agente
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36'
    }    
    # url
    start_urls = ['https://www.elempleo.com/co/ofertas-empleo/medellin']


    # Funcion para extraer la información
    def parse(self, response):
        # Selectores: Clase de scrapy para extraer datos
        sel = Selector(response) 

        # X Path del contenido de empleos 
        preguntas = sel.xpath('//div[@class="result-list js-result-list js-results-container"]/div') 
               
        i = 0
        for pregunta in preguntas:
            item = ItemLoader(Pregunta(), pregunta) # Instancio mi ITEM con el selector en donde estan los datos para llenarlo
            # Lleno las propiedades de mi ITEM a traves de expresiones XPATH a buscar dentro del selector "pregunta"
            item.add_value('id', i)
            item.add_xpath('fecha', './/span[@class="info-publish-date pull-right js-offer-date"]/text()')
            item.add_xpath('cargo', './/h2/a/text()') 
            item.add_xpath('salario', './/span[@class="text-primary info-salary js-offer-salary"]/text()')
            item.add_xpath('empresa', './/span[@class="info-company-name js-offer-company"]/text()')
            item.add_value('id', i)
            i += 1
            yield item.load_item() # Hago Yield de la informacion para que se escriban los datos en el archivo
            

In [3]:
#Ejecucion Spider
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)',
    'FEED_FORMAT':'json',                                 # Used for pipeline 2
    'FEED_URI': 'empleos.json'                        # Used for pipeline 2
})

process.crawl(QuotesSpider)
process.start()

2021-09-05 14:02:10 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-09-05 14:02:10 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Windows-10-10.0.19041-SP0
2021-09-05 14:02:10 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-09-05 14:02:10 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, '
               'like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 '
               'Safari/537.36'}
2021-09-05 14:02:10 [scrapy.extensions.telnet] INFO: Telnet Password: f1c7960ac9b677b3
2021-09-05 14:02:10 [py.warnings] WARNING: C:\ProgramData\Anaconda3\lib\site-packages\scrapy\extensions\feedexport.py:247: ScrapyDeprecationWarning:

In [4]:
with open('empleos.json', 'r', encoding = 'utf-8') as f:
    content = f.read()
    invalid_tags = ['\\r', '\\n', '<', '>', '-', ';']
    for invalid_tag in invalid_tags:
        content = content.replace(invalid_tag, '')
    content = content.replace('&u', 'ü')

In [5]:
import json
import pandas as pd
import re
from pandas.io.json import json_normalize
with open('empleos.json') as data_file:    
    d = json.load(data_file)
    df = json_normalize(d)

2021-09-05 14:02:13 [py.warnings] WARNING: <ipython-input-5-704f3d931823>:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(d)



In [18]:
df['cargo']=df['cargo'].str.join('').str.replace(r"\r\n", "")
df['cargo']=df['cargo'].str.join('').str.replace("'", "")
df['cargo']=df['cargo'].str.strip()
df['fecha']=df['fecha'].str.join('').str.replace(r"\r\n", "")
df['fecha']=df['fecha'].str.join('').str.replace("'", "")
df['fecha']=df['fecha'].str.join('').str.replace("Publicado", "")
df['fecha']=df['fecha'].str.strip()
df['fecha']= pd.to_datetime(df['fecha'])
df['salario']=df['salario'].str.join('').str.replace(r"\r\n", "")
df['salario']=df['salario'].str.join('').str.replace("'", "")
df['salario']=df['salario'].str.strip()
df['empresa']=df['empresa'].str.join('').str.replace(r"\r\n", "")
df['empresa']=df['empresa'].str.join('').str.replace("'", "")
df['empresa']=df['empresa'].str.strip()

In [14]:
df.to_excel("empleos.xlsx")  